In [19]:
import pandas as pd
import numpy as np
import csv

import warnings
warnings.filterwarnings('ignore')

from tqdm import tqdm

from datetime import datetime

d = datetime.today().strftime('%Y%m%d-%H%M%S')

data_dir = '/work/users/k/4/k4thryn/Repos/EpSampling/data/'

### <font color=blue> 1) County df with ground truth covid deaths.

In [5]:
df_counties.Fips.unique()

array([53061., 17031.,  6059., ..., 15005., 69100., 48999.])

In [14]:
df_counties = pd.read_csv(f'{data_dir}nytimes/us-counties.csv')
df_counties.columns = df_counties.columns.str.capitalize()
df_counties.drop(['Cases'],axis=1,inplace=True)
print(len(df_counties))
df_counties.dropna(inplace=True)
print(len(df_counties))

# df['Fips'] = df_counties['Fips'].astype('int64').astype('str')

2502832
2421549


In [ ]:
## Make list of dfs because everything in one df is too big.

all_states = list(df_counties.State.unique())

county_dfs = {}

for state in tqdm(all_states, total=len(all_states)):
    df = df_counties[df_counties.State==state]
    df['Fips'] = df['Fips'].astype('int64').astype('str')
#     display(df)
    county_dfs[state] = df
#     break
#     county_dfs[state] = df_counties[df_counties.State==state]

 55%|█████▍    | 29/53 [00:04<00:03,  6.39it/s]

In [13]:
county_dfs['Illinois'].dtypes



Date       object
County     object
State      object
Fips      float64
Deaths    float64
dtype: object

In [ ]:
dfff = 

,Date,County,State,Fips,Cases,Deaths
0,2020-01-21,Snohomish,Washington,53061.0,1,0.0
1,2020-01-22,Snohomish,Washington,53061.0,1,0.0
2,2020-01-23,Snohomish,Washington,53061.0,1,0.0
3,2020-01-24,Cook,Illinois,17031.0,1,0.0
4,2020-01-24,Snohomish,Washington,53061.0,1,0.0
...,...,...,...,...,...,...
2502827,2022-05-13,Sweetwater,Wyoming,56037.0,11088,126.0
2502828,2022-05-13,Teton,Wyoming,56039.0,10074,16.0
2502829,2022-05-13,Uinta,Wyoming,56041.0,5643,39.0
2502830,2022-05-13,Washakie,Wyoming,56043.0,2358,44.0


### <font color=blue> 2) Compute naive deaths for each county. 

In [4]:
## Get fips
state_to_fips = pd.read_csv('constants/state_fips.csv')
state_to_fips.rename({'FIPS':'State_fips'},axis=1,inplace=True)

## Add fips to forecast table.
df = pd.read_csv(f'{data_dir}covidhub_ensemble_1wkcum_point_20240911-114640.csv')
df.columns = df.columns.str.capitalize()
df.rename({'Location':'State_fips'},axis=1,inplace=True)

df = df.merge(state_to_fips, on='State_fips')

## Remove and rename columns ...
df_states = df.drop(['Target','Forecast_date'], axis=1, errors='ignore')
df_states.rename({'Target_end_date':'Date', 'Value':'COVIDhubEns_deaths'}, axis=1, inplace=True)
df_states

,State_fips,Date,COVIDhubEns_deaths,State,Postal
0,1,2020-04-18,152.238,Alabama,AL
1,1,2020-05-02,287.067,Alabama,AL
2,1,2020-05-09,350.653,Alabama,AL
3,1,2020-05-16,470.125,Alabama,AL
4,1,2020-05-23,593.986,Alabama,AL
...,...,...,...,...,...
7996,78,2023-02-11,129.000,Virgin Islands,VI
7997,78,2023-02-18,129.000,Virgin Islands,VI
7998,78,2023-02-25,131.000,Virgin Islands,VI
7999,78,2023-03-04,130.000,Virgin Islands,VI


### <font color=blue> Merge dataframes.

In [7]:
## Only need dates for counties that we have for states ... 

dates = list(df_states.Date.unique())

In [8]:
dff_counties = df_counties[df_counties.Date.isin(dates)]
dff_counties

,Date,County,State,Fips,Cases,Deaths
67364,2020-04-18,Autauga,Alabama,1001.0,25,2.0
67365,2020-04-18,Baldwin,Alabama,1003.0,109,2.0
67366,2020-04-18,Barbour,Alabama,1005.0,18,0.0
67367,2020-04-18,Bibb,Alabama,1007.0,26,0.0
67368,2020-04-18,Blount,Alabama,1009.0,20,0.0
...,...,...,...,...,...,...
2483280,2022-05-07,Sweetwater,Wyoming,56037.0,11075,126.0
2483281,2022-05-07,Teton,Wyoming,56039.0,10047,16.0
2483282,2022-05-07,Uinta,Wyoming,56041.0,5635,39.0
2483283,2022-05-07,Washakie,Wyoming,56043.0,2356,44.0


In [ ]:
dff = dff_counties.merge(df_states, on='State')
dff